In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from local_settings import postgresql as settings


In [3]:
import os
print(os.getcwd())

d:\Daniel\data_science\portfolio_projects\food_carbon_fp


In [4]:
def get_engine(user, passwd, host, port, db):
    url = f"mssql+pyodbc://{user}:{passwd}@{host},{port}/{db}?driver=ODBC+Driver+17+for+SQL+Server"
    engine = create_engine(url, pool_size= 50, echo= False)

    return engine

In [5]:
engine = get_engine(
    user="sa",
    passwd="10425573689",
    host="localhost",
    port=1433,
    db="food_carbon_footprint_2018"
)

In [6]:
from sqlalchemy import inspect
inspector = inspect(engine)
print(inspector.get_table_names())

['food_cfp', 'food_cfp_2018']
